In [1]:
import numpy as np
import random

In [76]:
def polynomial(x: float) -> float:
    y=1.5 * x**2 - 2 * x + 5
    return   y

x = np.linspace(-20, 20, 100)
y = polynomial(x) + np.random.normal(0, 1, 100)

In [77]:
class Indv:
    def __init__(self):
        self.genes = np.random.normal(0, 1, 3)  
        self.fitness = 0

    def calculate_fitness(self):
        predictions = self.genes[0] * X**2 + self.genes[1] * X + self.genes[2]
        self.fitness = -np.mean((Y - predictions)**2)

In [78]:
class GA:
    def __init__(self, population_size: int = 100):
        self.population = [Individual() for _ in range(population_size)]

    def evolve(self, mutation_rate: float, crossover_rate: float):
        for ind in self.population:
            ind.calculate_fitness()

        self.population.sort(key=lambda x: x.fitness, reverse=True)
        new_population = self.population[:2]  # Elitism

        while len(new_population) < len(self.population):
            if random.random() < crossover_rate:
                parent1 = self.tournament_selection()
                parent2 = self.tournament_selection()
                child = Individual()
                child.genes = np.where(np.random.rand(3) < 0.5, parent1.genes, parent2.genes)
                new_population.append(child)
            else:
                new_population.append(random.choice(self.population[:10]))

        for ind in new_population[2:]:
            if random.random() < mutation_rate:
                ind.genes += np.random.normal(0, 0.1 / (1 + np.exp(-ind.fitness)), 3)  

        self.population = new_population

    def tournament_selection(self, k=3):
        selected = random.sample(self.population, k)
        return max(selected, key=lambda ind: ind.fitness)

In [79]:
class Arm:
    def __init__(self, mutation_rate: float, crossover_rate: float):
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.value = 0
        self.pulls = 0

In [80]:
class KArmBandit:
    def __init__(self, arms: List[Arm]):
        self.arms = arms

    def softmax(self, values, temperature=1.0):
        values = np.array(values)  
        exp_values = np.exp(values / temperature)
        return exp_values / np.sum(exp_values)

    def select_arm(self, temperature: float) -> Arm:
        values = [arm.value for arm in self.arms]
        probabilities = self.softmax(values, temperature)
        selected_index = np.random.choice(len(self.arms), p=probabilities)
        return self.arms[selected_index]

    def update_arm(self, arm: Arm, reward: float):
        arm.pulls += 1
        arm.value += (reward - arm.value) / arm.pulls

In [81]:
arms = [
    Arm(mutation_rate=0.05, crossover_rate=0.7),
    Arm(mutation_rate=0.1, crossover_rate=0.6),
    Arm(mutation_rate=0.2, crossover_rate=0.5),
    Arm(mutation_rate=0.3, crossover_rate=0.8),
]

bandit = KArmBandit(arms)
temperature = 1.0  

def simulate_genetic_algorithm(mutation_rate, crossover_rate):
    return np.random.normal(loc=5 * mutation_rate + 2 * crossover_rate, scale=1)

for trial in range(100):
    selected_arm = bandit.select_arm(temperature)

    reward = simulate_genetic_algorithm(selected_arm.mutation_rate, selected_arm.crossover_rate)

    bandit.update_arm(selected_arm, reward)

best_arm = max(arms, key=lambda arm: arm.value)
print(f"Best mutation rate: {best_arm.mutation_rate}, Best crossover rate: {best_arm.crossover_rate}")


Best mutation rate: 0.3, Best crossover rate: 0.8
